# 앙상블: Boosting

1. XGboost를 적용해본다.
2. 다른 알고리즘들과 성능을 비교해본다.

## 1. 데이터 준비

### (1) 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

: 

### (2) 데이터 업로드

In [2]:
# mobile data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv"
data = pd.read_csv(path)
data.drop(['id', 'REPORTED_USAGE_LEVEL','OVER_15MINS_CALLS_PER_MONTH'], axis = 1, inplace = True)
data.rename(columns = {'HANDSET_PRICE':'H_PRICE',
                       'AVERAGE_CALL_DURATION':'DURATION',
                       'REPORTED_SATISFACTION':'SATISFACTION',
                       'CONSIDERING_CHANGE_OF_PLAN':'CHANGE'}
            , inplace = True)
data['CHURN'] = np.where(data['CHURN'] == 'LEAVE', 1, 0)  # XGBoost는 Label이 1,0 이어야 함
data.head()

| 변수 명 | 내용 | 구분 |
|----|----|----|
| COLLEGE | 대학졸업 여부(1,0) - 범주 | |
| INCOME | 연 수입액(달러) | |
| OVERAGE | 월 초과사용 시간(분) | |
| LEFTOVER | 월 사용 잔여시간비율(%) | |
| HOUSE | 집 가격(달러) | |
| HANDSET_PRICE | 핸드폰 가격(달러) | |
| AVERAGE_CALL_DURATION | 평균 통화시간(분) | |
| REPORTED_SATISFACTION | 만족도 설문('very_unsat', 'unsat', 'avg', 'sat', 'very_sat' ) - 범주 | |
| CONSIDERING_CHANGE_OF_PLAN | 변경 계획 설문('never_thought', 'no', 'perhaps', 'considering', 'actively_looking_into_it') - 범주 | |
| **CHURN** | 이탈여부(1 : 이탈, 0 : 잔류) | **Target** |

In [3]:
# 데이터분할1
target = 'CHURN'
x = data.drop(target, axis=1)
y = data.loc[:, target]

# 가변수화
dumm_cols = ['SATISFACTION','CHANGE']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

# 데이터 분할2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.5, random_state = 20)

## 2. 모델링
* Boosting 원리를 이용한 알고리즘 : XGB, LightGBM

In [4]:
# 1) 함수 불러오기
from xgboost import XGBClassifier, plot_tree
from sklearn.metrics import *

In [5]:
# 2) 모델 선언
model = XGBClassifier(n_estimators = 5, max_depth = 3)

In [6]:
# 3) 학습
model.fit(x_train, y_train)

In [7]:
# 4) 예측
pred = model.predict(x_val)

In [8]:
# 5) 평가
print(classification_report(y_val, pred))

## 3. XGBoost에 대해서...

### 1) 모델 시각화
xgboost 자체 plot_tree 함수를 제공합니다.
* `plot_tree(model, num_trees = 0)`
    * num_trees : 전체 트리 5개짜리 모델이므로 각각 0~4까지 인덱스로 조회해 볼 수 있다.

In [9]:
plt.rcParams['figure.figsize'] = 20,20  # 그래프 크기 설정

In [10]:
plot_tree(model, num_trees = 0)
plt.show()

In [11]:
plot_tree(model, num_trees = 4)
plt.show()

### 2) 변수 중요도

In [12]:
# 변수 중요도
print(x_train.columns)
print(model.feature_importances_)

* 변수중요도 그래프 그리기 함수 만들기

In [13]:
def plot_feature_importance(importance, names):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df.reset_index(drop=True, inplace = True)

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.grid()

In [14]:
plot_feature_importance(model.feature_importances_, x_train.columns)

## 4. 하이퍼파라미터 변화에 따른 성능 추세

### (1) n_estimators
* 1~150까지 증가시켜가며 성능 추세 살펴보기

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
grid_param = {'n_estimators':range(1,100,2)}
model = XGBClassifier()
model_gs = GridSearchCV(model, grid_param, cv = 5)
model_gs.fit(x_train, y_train)

In [17]:
result = pd.DataFrame(model_gs.cv_results_)

In [18]:
# 이 중에서 하이퍼파라미터 값에 따른 성능을 별도로 저장합시다.
temp = result.loc[:, ['param_n_estimators','mean_test_score']]
temp.head()

In [19]:
# 이를 차트로 그려봅시다.
plt.figure(figsize = (8,6))
sns.lineplot(x = 'param_n_estimators', y = 'mean_test_score', data = temp )
plt.grid()
plt.show()

### (2) learning rate
* 0.01 ~ 1까지 증가시켜가며 성능 추세 살펴보기

In [20]:
grid_param = {'learning_rate': np.linspace(0.02, 1, 50)}
model = XGBClassifier()
model_gs = GridSearchCV(model, grid_param, cv = 5)
model_gs.fit(x_train, y_train)

In [21]:
result = pd.DataFrame(model_gs.cv_results_)

In [22]:
# 이 중에서 하이퍼파라미터 값에 따른 성능을 별도로 저장합시다.
temp = result.loc[:, ['param_learning_rate','mean_test_score']]
temp.head()

In [23]:
# 이를 차트로 그려봅시다.
plt.figure(figsize = (8,6))
sns.lineplot(x = 'param_learning_rate', y = 'mean_test_score', data = temp )
plt.grid()
plt.show()

### (3) n_estimators + learning rate
* 이번에는 두 하이퍼파라미터를 동시에 조절하며 추세를 살펴보겠습니다.
* n_estimators : 10, 50, 100
* learning_rate : 0.01 ~ 0.3까지 0.01씩 증가

In [24]:
grid_param = {'learning_rate':np.linspace(0.01,0.3,30),
              'n_estimators':[10,50,100]}
model = XGBClassifier()
model_gs = GridSearchCV(model, grid_param, cv = 5)
model_gs.fit(x_train, y_train)

In [25]:
result = pd.DataFrame(model_gs.cv_results_)

In [26]:
# 이 중에서 하이퍼파라미터 값에 따른 성능을 별도로 저장합시다.
temp = result.loc[:, ['param_n_estimators', 'param_learning_rate','mean_test_score']]
temp.head()

In [27]:
# 이를 차트로 그려봅시다.
plt.figure(figsize = (8,6))
sns.lineplot(x = 'param_learning_rate', y = 'mean_test_score', data = temp, hue = 'param_n_estimators')
plt.grid()
plt.show()

## 5. 복습문제
* 카시트 데이터를 이용하여 랜덤포레스트 모델을 생성해 봅시다.
    * 카시트 판매량 예측 : 회귀
    * 기본 전처리
    * 알고리즘 : **XGBRegressor** (회귀 모델)
    * 모델 튜닝 : Grid Search

In [28]:
from xgboost import XGBRegressor

### (1) data loading

In [29]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv'

data = pd.read_csv(path)  # csv 파일을 불러올때, 지정한 칼럼의 데이터만 가져오기
data.head()

| 변수명 | 설명 | 구분 |
|----|----|----|
| Sales | 각 지역 판매량(단위 : 1000개) | Target |
| CompPrice | 지역별 경쟁사 판매가격(달러) | feature |
| Income | 가구당 평균 소득액(1000달러) | feature |
| Advertising | 각 지역, 회사의 광고 예산(1000달러) | feature |
| Population | 지역 인구수(단위 : 1000명) | feature |
| Price | 자사 지역별 판매가격(달러) | feature |
| ShelveLoc | 진열상태(범주 : Bad, Medium, Good) | feature |
| Age | 지역 인구의 평균 연령 | feature |
| Education | 교육수준(범주 : 10~18) | feature |
| Urban | 매장이 도심에 있는지 여부(범주 : Yes, No) | feature |
| US | 매장이 미국에 있는지 여부(범주 : Yes, No) | feature |

### (2) 데이터분할1 : x, y 나누기

In [30]:
target = 'Sales'
x = data.drop(target, axis=1)
y = data.loc[:, target]

### (3) 가변수화

In [31]:
cat_cols = ['ShelveLoc', 'US','Urban']

In [32]:
# 첫번째 가변수 제거
x = pd.get_dummies(x, columns=cat_cols, drop_first=True)
x.head()

### (4) 데이터분할2 : train : validation 나누기

In [33]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (5) 모델링 : 튜닝
* 성능 튜닝을 수행해 봅시다.
* 하이퍼파라미터 범위
    * cv = 3
    * grid 파라미터
        * max_depth : 3 ~ 8 사이에서 3개 선택
        * n_estimators : 5 ~ 100 사이에서 5개 선택
        * learning_rate :0.01 ~ 0.2 사이에서 3개 선택
* 튜닝 후 검증셋으로 예측하고, RMSE, MAE, MAPE로 평가해 봅시다.

In [34]:
from sklearn.model_selection import GridSearchCV

# 1. 파라미터 그리드 생성
# max_depth: 3~8 사이 3개 [3, 5, 8]
# n_estimators: 5~100 사이 5개 [5, 25, 50, 75, 100]
# learning_rate: 0.01~0.2 사이 3개 [0.01, 0.1, 0.2]

param_grid = {
    'max_depth': [3, 5, 8],
    'n_estimators': [5, 25, 50, 75, 100],
    'learning_rate': [0.01, 0.1, 0.2]
}

# 2. 모델 선언 (회귀 모델)
model = XGBRegressor(objective='reg:squarederror')

In [35]:
# 3. Grid Search 모델 선언 및 학습 (cv=3)
model_gs = GridSearchCV(model, param_grid, cv=3)
model_gs.fit(x_train, y_train)

In [36]:
# 4. 최적 파라미터 및 점수 확인
print("Best Parameters:", model_gs.best_params_)
print("Best Score:", model_gs.best_score_)

# 5. 예측
pred = model_gs.predict(x_val)

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 6. 평가 (RMSE, MAE, MAPE)
print('RMSE :', mean_squared_error(y_val, pred, squared = True))
print('MAE  :', mean_absolute_error(y_val, pred))
print('MAPE :', mean_absolute_percentage_error(y_val, pred))